<a href="https://colab.research.google.com/github/Peby1407/deep-learning-challenge/blob/main/AlphabetSoupCharity_Optimization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Preprocessing

In [57]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf

#  Import and read the charity_data.csv.
import pandas as pd
application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [58]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df = application_df.drop(columns=["EIN", "NAME"])
application_df

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1
...,...,...,...,...,...,...,...,...,...,...
34294,T4,Independent,C1000,ProductDev,Association,1,0,N,5000,0
34295,T4,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
34296,T3,CompanySponsored,C2000,Preservation,Association,1,0,N,5000,0
34297,T5,Independent,C3000,ProductDev,Association,1,0,N,5000,1


In [59]:
# Determine the number of unique values in each column.
unique_counts = application_df.nunique()

print(unique_counts)

APPLICATION_TYPE            17
AFFILIATION                  6
CLASSIFICATION              71
USE_CASE                     5
ORGANIZATION                 4
STATUS                       2
INCOME_AMT                   9
SPECIAL_CONSIDERATIONS       2
ASK_AMT                   8747
IS_SUCCESSFUL                2
dtype: int64


In [60]:
# Look at APPLICATION_TYPE value counts for binning
application_df['APPLICATION_TYPE'].value_counts()

T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: APPLICATION_TYPE, dtype: int64

In [61]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
application_types_to_replace = ['T9','T13','T12','T2','T25','T14','T29','T15','T17']

# Replace in dataframe
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure binning was successful
application_df['APPLICATION_TYPE'].value_counts()

T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: APPLICATION_TYPE, dtype: int64

In [62]:
# Look at CLASSIFICATION value counts for binning
application_df['CLASSIFICATION'].value_counts()

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C4120        1
C8210        1
C2561        1
C4500        1
C2150        1
Name: CLASSIFICATION, Length: 71, dtype: int64

In [63]:
# You may find it helpful to look at CLASSIFICATION value counts >1
value_counts = application_df['CLASSIFICATION'].value_counts()

value_counts_greater_than_1 = value_counts[value_counts > 1]

value_counts_greater_than_1

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
C2800       95
C7100       75
C1300       58
C1280       50
C1230       36
C1400       34
C7200       32
C2300       32
C1240       30
C8000       20
C7120       18
C1500       16
C1800       15
C6000       15
C1250       14
C8200       11
C1238       10
C1278       10
C1235        9
C1237        9
C7210        7
C2400        6
C1720        6
C4100        6
C1257        5
C1600        5
C1260        3
C2710        3
C0           3
C3200        2
C1234        2
C1246        2
C1267        2
C1256        2
Name: CLASSIFICATION, dtype: int64

In [64]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
value_counts_smaller_than_778 = value_counts[value_counts < 778]
classifications_to_replace = value_counts_smaller_than_778.index.tolist()

# Replace in dataframe
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")

# Check to make sure binning was successful
application_df['CLASSIFICATION'].value_counts()

C1000    17326
C2000     6074
C1200     4837
Other     2261
C3000     1918
C2100     1883
Name: CLASSIFICATION, dtype: int64

In [65]:
# Convert categorical data to numeric with `pd.get_dummies`
columns_to_convert = [
    'APPLICATION_TYPE', 'AFFILIATION', 'CLASSIFICATION',
    'USE_CASE', 'ORGANIZATION', 'INCOME_AMT', 'SPECIAL_CONSIDERATIONS'
]

dummies = pd.get_dummies(application_df, columns=columns_to_convert)

In [66]:
dummies = dummies.drop(columns='IS_SUCCESSFUL')
dummies

,STATUS,ASK_AMT,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,APPLICATION_TYPE_T7,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,1,5000,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
1,1,108590,0,0,0,1,0,0,0,0,...,1,0,0,0,0,0,0,0,1,0
2,1,5000,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,1,0
3,1,6692,0,0,0,1,0,0,0,0,...,0,1,0,0,0,0,0,0,1,0
4,1,142590,0,0,0,1,0,0,0,0,...,0,0,1,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34294,1,5000,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,1,0
34295,1,5000,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,1,0
34296,1,5000,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
34297,1,5000,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,1,0


In [67]:
application_df = pd.concat([application_df, dummies], axis=1)
application_df = application_df.drop(columns=columns_to_convert)
application_df

,STATUS,ASK_AMT,IS_SUCCESSFUL,STATUS,ASK_AMT,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,1,5000,1,1,5000,0,1,0,0,0,...,0,0,0,0,0,0,0,0,1,0
1,1,108590,1,1,108590,0,0,0,1,0,...,1,0,0,0,0,0,0,0,1,0
2,1,5000,0,1,5000,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
3,1,6692,1,1,6692,0,0,0,1,0,...,0,1,0,0,0,0,0,0,1,0
4,1,142590,1,1,142590,0,0,0,1,0,...,0,0,1,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34294,1,5000,0,1,5000,0,0,0,0,1,...,0,0,0,0,0,0,0,0,1,0
34295,1,5000,0,1,5000,0,0,0,0,1,...,0,0,0,0,0,0,0,0,1,0
34296,1,5000,0,1,5000,0,0,0,1,0,...,0,0,0,0,0,0,0,0,1,0
34297,1,5000,1,1,5000,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0


In [68]:
# Split our preprocessed data into our features and target arrays
X = application_df.drop(columns='IS_SUCCESSFUL')
y = application_df['IS_SUCCESSFUL']

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=10)

In [70]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
scaler.fit(X_train)

# Scale the data
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

## Compile, Train and Evaluate the Model

In [95]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=80, input_dim=45, activation='relu'))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=80, activation='relu'))


# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

# Check the structure of the model
nn.summary()

Model: "sequential_18"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_68 (Dense)            (None, 80)                3680      
                                                                 
 dense_69 (Dense)            (None, 80)                6480      
                                                                 
 dense_70 (Dense)            (None, 1)                 81        
                                                                 
Total params: 10,241
Trainable params: 10,241
Non-trainable params: 0
_________________________________________________________________


In [75]:
# Compile the model
nn.compile(
    loss = 'binary_crossentropy',
    optimizer='adam',
    metrics=['accuracy']
)

In [76]:
# Train the model
nn.fit(X_train_scaled, y_train, epochs=100)

Epoch 1/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5712 - accuracy: 0.7191
Epoch 2/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5545 - accuracy: 0.7312
Epoch 3/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5526 - accuracy: 0.7319
Epoch 4/100
804/804 [==============================] - 2s 3ms/step - loss: 0.5493 - accuracy: 0.7325
Epoch 5/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5496 - accuracy: 0.7332
Epoch 6/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5484 - accuracy: 0.7334
Epoch 7/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5475 - accuracy: 0.7343
Epoch 8/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5468 - accuracy: 0.7358
Epoch 9/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5468 - accuracy: 0.7352
Epoch 10/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5467 - accura

In [77]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5608 - accuracy: 0.7262 - 464ms/epoch - 2ms/step
Loss: 0.5608464479446411, Accuracy: 0.7261807322502136


In [78]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=80, input_dim=45, activation='relu'))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=80, activation='relu'))

nn.add(tf.keras.layers.Dense(units=80, activation='relu'))

nn.add(tf.keras.layers.Dense(units=80, activation='relu'))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

# Check the structure of the model
nn.summary()

Model: "sequential_14"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_47 (Dense)            (None, 80)                3680      
                                                                 
 dense_48 (Dense)            (None, 80)                6480      
                                                                 
 dense_49 (Dense)            (None, 80)                6480      
                                                                 
 dense_50 (Dense)            (None, 80)                6480      
                                                                 
 dense_51 (Dense)            (None, 1)                 81        
                                                                 
Total params: 23,201
Trainable params: 23,201
Non-trainable params: 0
_________________________________________________________________


In [79]:
# Compile the model
nn.compile(
    loss = 'binary_crossentropy',
    optimizer='adam',
    metrics=['accuracy']
)

In [80]:
# Train the model
nn.fit(X_train_scaled, y_train, epochs=100)

Epoch 1/100
804/804 [==============================] - 4s 3ms/step - loss: 0.5695 - accuracy: 0.7217
Epoch 2/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5548 - accuracy: 0.7299
Epoch 3/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5515 - accuracy: 0.7323
Epoch 4/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5504 - accuracy: 0.7339
Epoch 5/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5481 - accuracy: 0.7348
Epoch 6/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5475 - accuracy: 0.7352
Epoch 7/100
804/804 [==============================] - 2s 3ms/step - loss: 0.5470 - accuracy: 0.7361
Epoch 8/100
804/804 [==============================] - 2s 3ms/step - loss: 0.5467 - accuracy: 0.7350
Epoch 9/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5459 - accuracy: 0.7352
Epoch 10/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5454 - accura

In [81]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5814 - accuracy: 0.7259 - 495ms/epoch - 2ms/step
Loss: 0.5813749432563782, Accuracy: 0.7259474992752075


In [91]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=100, input_dim=45, activation='relu'))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=100, activation='relu'))

nn.add(tf.keras.layers.Dense(units=100, activation='relu'))

nn.add(tf.keras.layers.Dense(units=100, activation='relu'))

nn.add(tf.keras.layers.Dense(units=100, activation='relu'))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

# Check the structure of the model
nn.summary()

Model: "sequential_17"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_62 (Dense)            (None, 100)               4600      
                                                                 
 dense_63 (Dense)            (None, 100)               10100     
                                                                 
 dense_64 (Dense)            (None, 100)               10100     
                                                                 
 dense_65 (Dense)            (None, 100)               10100     
                                                                 
 dense_66 (Dense)            (None, 100)               10100     
                                                                 
 dense_67 (Dense)            (None, 1)                 101       
                                                                 
Total params: 45,101
Trainable params: 45,101
Non-tra

In [92]:
# Compile the model
nn.compile(
    loss = 'binary_crossentropy',
    optimizer='adam',
    metrics=['accuracy']
)

In [93]:
# Train the model
nn.fit(X_train_scaled, y_train, epochs=100)

Epoch 1/100
804/804 [==============================] - 4s 3ms/step - loss: 0.5691 - accuracy: 0.7221
Epoch 2/100
804/804 [==============================] - 2s 3ms/step - loss: 0.5549 - accuracy: 0.7303
Epoch 3/100
804/804 [==============================] - 2s 3ms/step - loss: 0.5519 - accuracy: 0.7317
Epoch 4/100
804/804 [==============================] - 3s 3ms/step - loss: 0.5501 - accuracy: 0.7334
Epoch 5/100
804/804 [==============================] - 3s 4ms/step - loss: 0.5492 - accuracy: 0.7360
Epoch 6/100
804/804 [==============================] - 2s 3ms/step - loss: 0.5476 - accuracy: 0.7346
Epoch 7/100
804/804 [==============================] - 2s 3ms/step - loss: 0.5475 - accuracy: 0.7350
Epoch 8/100
804/804 [==============================] - 2s 3ms/step - loss: 0.5461 - accuracy: 0.7361
Epoch 9/100
804/804 [==============================] - 2s 3ms/step - loss: 0.5458 - accuracy: 0.7355
Epoch 10/100
804/804 [==============================] - 3s 4ms/step - loss: 0.5453 - accura

In [94]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 1s - loss: 0.5781 - accuracy: 0.7269 - 522ms/epoch - 2ms/step
Loss: 0.5781326293945312, Accuracy: 0.7268804907798767


In [87]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=100, input_dim=45, activation='tanh'))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=100, activation='tanh'))

nn.add(tf.keras.layers.Dense(units=100, activation='tanh'))



# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

# Check the structure of the model
nn.summary()

Model: "sequential_16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_58 (Dense)            (None, 100)               4600      
                                                                 
 dense_59 (Dense)            (None, 100)               10100     
                                                                 
 dense_60 (Dense)            (None, 100)               10100     
                                                                 
 dense_61 (Dense)            (None, 1)                 101       
                                                                 
Total params: 24,901
Trainable params: 24,901
Non-trainable params: 0
_________________________________________________________________


In [88]:
# Compile the model
nn.compile(
    loss = 'binary_crossentropy',
    optimizer='adam',
    metrics=['accuracy']
)

In [89]:
# Train the model
nn.fit(X_train_scaled, y_train, epochs=100)

Epoch 1/100
804/804 [==============================] - 3s 2ms/step - loss: 0.5719 - accuracy: 0.7206
Epoch 2/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5580 - accuracy: 0.7288
Epoch 3/100
804/804 [==============================] - 2s 3ms/step - loss: 0.5541 - accuracy: 0.7311
Epoch 4/100
804/804 [==============================] - 3s 3ms/step - loss: 0.5517 - accuracy: 0.7318
Epoch 5/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5503 - accuracy: 0.7325
Epoch 6/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5485 - accuracy: 0.7334
Epoch 7/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5484 - accuracy: 0.7336
Epoch 8/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5475 - accuracy: 0.7343
Epoch 9/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5456 - accuracy: 0.7358
Epoch 10/100
804/804 [==============================] - 2s 3ms/step - loss: 0.5454 - accura

In [90]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 1s - loss: 0.5615 - accuracy: 0.7264 - 696ms/epoch - 3ms/step
Loss: 0.5615469217300415, Accuracy: 0.7264139652252197


In [86]:
# Export our model to HDF5 file
nn.save("model_challenge_21.h5")